<a href="https://colab.research.google.com/github/LeanDev-bit/Analisis-con-SQLAlchemy/blob/main/Analisis_Tendencias_Produccion_Energia_Global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Cómo han cambiado las tendencias de producción de energía global a lo largo del tiempo?

In [ ]:
import pandas as pd
import numpy as np

## Objetivos

Al final de este caso, deberías sentirte muy cómodo escribiendo tus propias funciones usando pandas y aplicándolas a conjuntos de datos completos. Entenderás cómo funcionan las funciones en Python, incluidas las funciones anónimas (usando la palabra clave lambda), y te sentirás cómodo analizando y manipulando grandes conjuntos de datos. También habrás adquirido experiencia explorando un conjunto de datos que está solo vagamente organizado y sobre el cual tienes muy poca información inicial.


## Introducción

**Contexto empresarial.** La producción, consumo, importación y exportación de electricidad a nivel global es un proceso complejo e interesante por varias razones. Cada país tiene que hacer un seguimiento de una gran cantidad de información para asegurarse de producir suficiente electricidad, y a la vez equilibrar estas necesidades con las implicaciones financieras a mediano plazo y las preocupaciones ambientales.

Eres un analista que trabaja en una organización no gubernamental (ONG) que informa sobre las tendencias energéticas globales. Tu departamento ha adquirido un archivo CSV grande, pero tus colegas están teniendo dificultades para extraer conocimientos relevantes de él usando Excel debido a su tamaño y formato. Aún peor, tiene miles de variables y no están seguros de cuáles son las interesantes. Por ello, te han asignado la responsabilidad de apoyar a los periodistas de tu equipo proporcionándoles datos y conocimientos que puedan convertir en informes escritos.

**Problema empresarial.**  Tu tarea es **desglosar los datos disponibles en archivos más pequeños, comprender la información disponible y extraer conocimientos clave para un próximo informe sobre los patrones de energía global.** En concreto, tu equipo quiere que respondas a las siguientes preguntas:

* ¿Cuánta energía se produce?
* ¿Cuánta energía se consume?
* ¿Cuánta energía se importa y se exporta?
* ¿Cuánta de esta energía es renovable?
* ¿Cómo están cambiando estas tendencias de producción, consumo, importación y exportación a lo largo del tiempo?

**Contexto analítico.** Los datos están almacenados en un gran archivo CSV que contiene información sobre la producción y consumo de energía por país y año. Vas a: 1) dividir los datos en archivos CSV resumidos para compartir con tus colegas; 2) manipular los datos para crear más categorías a partir de las columnas existentes; 3) encontrar los principales actores en diferentes categorías, incluyendo la exportación total de energía y la producción total por tipo (por ejemplo, nuclear); y finalmente 4) encontrar tendencias en los datos, como los países con el crecimiento más rápido en la producción de energía.

## Empezando con los datos de Estadísticas Internacionales de Energía

El archivo de datos que tienes está ubicado en data/all_energy_statistics.csv. Tus colegas te han informado que los datos provienen de data.un.org, pero no saben mucho más al respecto.

Han señalado específicamente que los datos son muy "estrechos". Aunque el archivo contiene datos sobre una amplia variedad de conceptos, como "Producción Total de Energía" hasta "Aditivos y Oxigenantes - Exportaciones", tiene muy pocas columnas.

Por lo general, cuando se trabaja con datos "amplios", podemos estar bastante seguros de que toda la información en la misma columna es comparable. En este caso, notarás una columna unit. No todos los datos numéricos de la columna quantity son directamente comparables. Por ejemplo, a veces el número en esta columna está definido en términos de "toneladas métricas, mil" y a veces en "kilovatios-hora, millón", conceptos evidentemente muy diferentes.

Como siempre, nuestro primer paso es leer los datos desde el disco y echar un vistazo a las primeras filas:

In [ ]:
import requests
import zipfile
import os

local_zip_path = "all_energy_statistics.zip"  # Guardar el ZIP localmente

# Descargar el archivo
response = requests.get("https://github.com/dhcastrog/Bootcamp_BG/raw/refs/heads/main/Reto4/all_energy_statistics.zip")
with open(local_zip_path, 'wb') as file:
    file.write(response.content)

print("ZIP descargado exitosamente.")

unzip_folder = 'unzipped_data'
os.makedirs(unzip_folder, exist_ok=True)

# Step 3: Extraer el archivo ZIP
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder)

print("Archivos extraídos exitosamente.")
csv_file_path = os.path.join(unzip_folder, 'all_energy_statistics.csv')

df = pd.read_csv(csv_file_path)

ZIP descargado exitosamente.
Archivos extraídos exitosamente.


In [ ]:
df

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates
...,...,...,...,...,...,...,...
1189477,Viet Nam,Electricity - total wind production,2012,"Kilowatt-hours, million",92.0,1.0,wind_electricity
1189478,Viet Nam,Electricity - total wind production,2011,"Kilowatt-hours, million",87.0,NaN,wind_electricity
1189479,Viet Nam,Electricity - total wind production,2010,"Kilowatt-hours, million",50.0,NaN,wind_electricity
1189480,Viet Nam,Electricity - total wind production,2009,"Kilowatt-hours, million",10.0,NaN,wind_electricity


Notarás que hay más demora de lo habitual al ejecutar la función read_csv. Este conjunto de datos tiene más de 1 millón de filas, por lo que lleva un tiempo cargarlo completamente en memoria. Desde las primeras filas, podemos obtener algunas ideas útiles:

* La columna category parece estar bien organizada. Todas las muestras que vemos están en minúsculas y se usan guiones bajos en lugar de espacios.
* La columna commodity_transaction parece ser una descripción más comprensible para los humanos. Podemos ver cómo incluye una descripción de la categoría (por ejemplo, "additives_and_oxygenates" coincide con "Aditivos y Oxigenantes" y "wind_electricity" coincide con "Electricidad - ....viento....").
* Vemos que el rango de year va al menos desde 1995 hasta 2014.
* Como mencionamos antes, tendremos que ser cuidadosos al comparar cantidades, ya que la columna unit podría cambiar el significado de la columna quantity.

Una buena primera pregunta es cuántos valores únicos hay en las siguientes columnas:

* `country_or_area`
* `commodity_transaction`
* `year`
* `category`

Vamos a averiguarlo:

In [ ]:
print(df.year.min())
print(df.year.max())
print("----------")
print("commodity_transaction")
print(df.commodity_transaction.unique())
print()
print("num unique values: ", len(df.commodity_transaction.unique()))
print()
print("----------")
print(df.category.unique())
print()
print("num unique values: ", len(df.category.unique()))
print()
print("---------------")
print(df.country_or_area.unique())
print()
print("num unique values: ", len(df.country_or_area.unique()))


1990
2014
----------
commodity_transaction
['Additives and Oxygenates - Exports' 'Additives and Oxygenates - Imports'
 'Additives and Oxygenates - Production' ...
 'White spirit and special boiling point industrial spirits - Transformation'
 'White spirit and special boiling point industrial spirits - Transformation in petrochemical plants'
 'Electricity - total wind production']

num unique values:  2452

----------
['additives_and_oxygenates' 'animal_waste' 'anthracite'
 'aviation_gasoline' 'bagasse' 'biodiesel' 'biogases' 'biogasoline'
 'bitumen' 'black_liquor' 'blast_furnace_gas' 'brown_coal_briquettes'
 'brown_coal' 'charcoal' 'coal_tar' 'coke_oven_coke' 'coking_coal'
 'conventional_crude_oil' 'direct_use_of_geothermal_heat'
 'direct_use_of_solar_thermal_heat'
 'electricity_net_installed_capacity_of_electric_power_plants' 'ethane'
 'falling_water' 'fuel_oil' 'fuelwood' 'gas_coke' 'gas_oil_diesel_oil'
 'gasoline_type_jet_fuel' 'gasworks_gas' 'geothermal' 'hard_coal' 'heat'
 'hydro'

Podemos ver que country_or_area tiene 243 valores únicos, más que los 195 reconocidos oficialmente, porque esta lista incluye algunos países que ya no existen, como la URSS, así como áreas como las Pesquerías Antárticas, que no son países formales.

Como era de esperarse, la columna category está bien estandarizada y divide cada fila en una de 71 categorías únicas, mientras que la columna commodity_transaction es un poco más caótica y consta de 2452 valores únicos.

En términos de tiempo, nuestros datos abarcan desde 1990 hasta 2014, es decir, 25 años en total.

Ten en cuenta que el resultado de unique() se trunca automáticamente para listas grandes, con un ... insertado para indicar esto.

Dado que la columna commodity_transaction es un poco caótica, tendremos que corregirla un poco. Creemos una copia de nuestro DataFrame antes de empezar a modificarlo para poder referirnos a los valores originales si es necesario.

In [ ]:
df_orig = df.copy()

Lo primero que notamos sobre la columna commodity_transaction es que usa guiones (-) como separadores. También podemos ver que utiliza letras en minúsculas y mayúsculas, lo cual a menudo hace más difícil el análisis si vamos a hacer coincidencias de cadenas (por ejemplo, encontrar la palabra "producción", que podría no coincidir con descripciones que usan "Producción").

Empecemos por convertir todas las descripciones a minúsculas. En el caso anterior, aprendiste a hacer esto creando una lista separada, recorriendo el DataFrame y luego agregando todos los elementos de la lista como una nueva columna. Podríamos lograr lo que queremos de la siguiente manera:

In [ ]:
%%time
clean_transaction_list = []

for item in df['commodity_transaction']:
    item = item.lower()
    clean_transaction_list.append(item)

df['clean_transaction'] = clean_transaction_list

CPU times: user 779 ms, sys: 115 ms, total: 894 ms
Wall time: 1.9 s


In [ ]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


Agregamos %%time al inicio de nuestra celda para que Jupyter muestre información sobre el tiempo que tomó ejecutar esa celda. Podemos ver que recorrer nuestro DataFrame y agregar la columna tomó casi 1 segundo. También tomó 5 líneas de código.

Como es muy común necesitar aplicar la misma operación en cada fila de un conjunto de datos, pandas proporciona un atajo para hacerlo. Puedes usar la función .apply() directamente en un DataFrame y pasarle una función para aplicarla a cada fila. Esto es más eficiente en dos sentidos:

* Requiere menos líneas de código, por lo que es más rápido escribirlo (y leerlo).
* `apply()` está optimizado para aprovechar características modernas de CPU como la vectorización, por lo que se ejecuta en menos tiempo.

Podemos lograr exactamente el mismo resultado que con nuestro bucle utilizando la función .apply() de la siguiente manera:

In [ ]:
%%time
df['clean_transaction2'] = df['commodity_transaction'].apply(str.lower)

CPU times: user 360 ms, sys: 151 ms, total: 511 ms
Wall time: 1.58 s


In [ ]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction,clean_transaction2
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports


Aquí podemos ver que .apply() se ejecutó aproximadamente el doble de rápido que la versión iterativa y produjo los mismos resultados (las columnas clean_transaction y clean_transaction2 son iguales).

## Preprocesamiento y pivotado de nuestros datos

Antes notamos que la columna commodity_transaction parecía usar guiones para separar diferentes conceptos en una sola columna. Hagamos un análisis adicional para ver si esto se cumple en todo el conjunto de datos.

### Ejercicio 1:

Averigua cuántas de las más de 2000 columnas únicas contienen:

- 0 guiones
- exactamente 1 guion
- más de 1 guion

**Pista:** Puedes usar el método count() incorporado en Python para contar las ocurrencias de un carácter en una cadena.

**Respuesta.**

In [ ]:
# Averigua cuántas de las más de 2000 columnas únicas contienen:
# 0 guiones
# exactamente 1 guion
# más de 1 guion
# Pista: Puedes usar el método count() incorporado en Python para contar las ocurrencias de un carácter en una cadena.
%%time
# obtener valores unicos de la columna comodity_transaction
commodity_transaction_unique = df.commodity_transaction.unique()

# countar veces que se repite el - guion
dash_count = [x.count('-') for x in commodity_transaction_unique]
# Crear un Pandas Series para obtener las veces que se repiten los guiones por tipos unicos
dash_count_series = pd.Series(dash_count)
dash_frequency = dash_count_series.value_counts()
dash_frequency

CPU times: user 171 ms, sys: 16.2 ms, total: 187 ms
Wall time: 581 ms


,count
1,1845
2,538
0,57
3,12


Podemos ver que la mayoría de las descripciones tienen exactamente un guion, lo que refuerza la idea de que la primera parte de la descripción antes del guion está relacionada con la category, mientras que el resto es más descriptivo. Deberíamos revisar más de cerca las descripciones con cero guiones, ya que solo hay 57.

### Ejercicio 2:

Escribe código para imprimir todas las descripciones con cero guiones. ¿Qué observas sobre estas?

**Respuesta.**

In [ ]:
zero_dash_frequency = df[df['clean_transaction'].str.count('-–') == 0]
zero_dash_frequency

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction,clean_transaction2
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
...,...,...,...,...,...,...,...,...,...
1189477,Viet Nam,Electricity - total wind production,2012,"Kilowatt-hours, million",92.0,1.0,wind_electricity,electricity - total wind production,electricity - total wind production
1189478,Viet Nam,Electricity - total wind production,2011,"Kilowatt-hours, million",87.0,NaN,wind_electricity,electricity - total wind production,electricity - total wind production
1189479,Viet Nam,Electricity - total wind production,2010,"Kilowatt-hours, million",50.0,NaN,wind_electricity,electricity - total wind production,electricity - total wind production
1189480,Viet Nam,Electricity - total wind production,2009,"Kilowatt-hours, million",10.0,NaN,wind_electricity,electricity - total wind production,electricity - total wind production


### Pasar nuestras propias funciones a apply()

Anteriormente, usamos la función incorporada str.lower() con apply() para aplicarla a cada fila de nuestro DataFrame. Ahora queremos limpiar los guiones largos (m-dashes) y convertir todo a minúsculas al mismo tiempo. Escribamos una función personalizada de Python para hacer ambas cosas y pasémosla a apply().

In [ ]:
def clean_transaction_description(transaction_description):
    """Convierte la entrada a minúsculas y reemplaza todos los guiones largos por guiones cortos"""
    clean = transaction_description.lower()
    clean = clean.replace("–", "-")
    return clean

# Ahora eliminemos las columnas anteriores para recrearlas con nuestra nueva función de limpieza:
df = df.drop(columns=['clean_transaction', 'clean_transaction2'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


In [ ]:
df['clean_transaction'] = df['commodity_transaction'].apply(clean_transaction_description)
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


Aquí usamos apply() nuevamente, pero esta vez pasamos nuestra propia función, que convirtió a minúsculas y reemplazó los guiones largos con guiones cortos.


**Usar funciones anónimas con lambda**

Además de funciones integradas y personalizadas, también podemos usar funciones anónimas con la palabra clave lambda. Veamos cómo lograr el mismo resultado usando lambda:

In [ ]:
df = df.drop(columns=['clean_transaction'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


In [ ]:
# Convierte a minúsculas y reemplaza guiones largos por guiones cortos en una sola línea
df['clean_transaction'] = df['commodity_transaction'].apply(lambda x: x.lower().replace("–", "-"))
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


Este código es funcionalmente equivalente al anterior, pero más conciso. En lugar de nombrar nuestra función, usamos lambda para declarar una función anónima que describe lo que hacer con cada valor. La ventaja es la concisión, pero la desventaja es que puede ser más difícil de leer y no se puede reutilizar sin volver a definirla.

### Extracción de las filas más interesantes

Inspeccionar manualmente más de 2000 descripciones únicas es difícil, pero sabemos que principalmente estamos interesados en:

* Import
* Export
* Total production
* Total demand or consumption
* Renewables

Podemos buscar palabras clave en las descripciones con un código similar al siguiente:

In [ ]:
[x for x in df['clean_transaction'].unique() if "import" in x]

['additives and oxygenates - imports',
 'anthracite - imports',
 'aviation gasoline - imports',
 'biodiesel - imports',
 'biogases - imports',
 'biogasoline - imports',
 'bitumen - imports',
 'brown coal briquettes - imports',
 'brown coal - imports',
 'charcoal - imports',
 'coal tar - imports',
 'coking coal - imports',
 'conventional crude oil - imports',
 'ethane - imports',
 'fuel oil - imports',
 'fuelwood - imports',
 'gas coke - imports',
 'gas oil/ diesel oil - imports',
 'gasoline-type jet fuel - imports',
 'gasworks gas - imports',
 'hard coal - imports',
 'heat - imports',
 'industrial waste - imports',
 'kerosene-type jet fuel - imports',
 'lignite - imports',
 'liquefied petroleum gas (lpg) - imports',
 'lubricants - imports',
 'motor gasoline - imports',
 'municipal wastes - imports',
 'naphtha - imports',
 'natural gas (including lng) - imports',
 'natural gas liquids - imports',
 'of which: biodiesel - imports',
 'of which: biogasoline - imports',
 'oil shale - imports

Esto nos proporciona una lista más manejable, y podemos ver que "electricity - imports" es probablemente un valor interesante. Podemos verificar esto en el conjunto de datos principal de la siguiente manera:

In [ ]:
df[df.clean_transaction == "electricity - imports"].head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
1108326,Afghanistan,Electricity - imports,2014,"Kilowatt-hours, million",3710.8,NaN,total_electricity,electricity - imports
1108327,Afghanistan,Electricity - imports,2013,"Kilowatt-hours, million",3615.2,NaN,total_electricity,electricity - imports
1108328,Afghanistan,Electricity - imports,2012,"Kilowatt-hours, million",3071.0,NaN,total_electricity,electricity - imports
1108329,Afghanistan,Electricity - imports,2011,"Kilowatt-hours, million",2732.0,NaN,total_electricity,electricity - imports
1108330,Afghanistan,Electricity - imports,2010,"Kilowatt-hours, million",1867.0,NaN,total_electricity,electricity - imports


### Ejercicio 3:

Usa el método anterior u otro que prefieras para explorar las descripciones de las transacciones y define una lista de Python con las 9 más interesantes. Estas deben cubrir los valores totales de importación, exportación, producción, demanda y producción de energía renovable.

**Respuesta.**

In [ ]:
keep_values = [
    'Electricity - Gross demand',
    'Electricity - Gross production',
    'Electricity - imports',
    'Electricity - exports',
    'Electricity - total hydro production',
    'Electricity - total wind production',
    'Electricity - total solar production',
    'Electricity - total geothermal production',
    'Electricity - total tide, wave production',
]

### Pivotando los valores interesantes en nuevas columnas

Por supuesto, ahora que hemos identificado las descripciones de transacciones más interesantes, probablemente deberíamos extraerlas de esa columna única en la que están atrapadas. Vamos a "pivotar" nuestros datos a un formato más útil, manteniendo cada uno de estos valores interesantes como nuevas columnas. Esto traduce nuestros datos de un formato bastante estrecho a uno más amplio.

Es posible que conozcas las "tablas dinámicas" de Excel. Si no es así, no te preocupes: te las encontrarás más adelante con más detalle. Pero si ya las conoces, reconocerás que esto es prácticamente lo mismo. Usaremos la función de pivotado en pandas. Por ahora, solo intenta comprender cómo funciona el siguiente código; no se espera que hagas esto por tu cuenta hasta que tengas más experiencia con pandas:

In [ ]:
# Conservaremos nuestros valores "interesantes" después de convertirlos en columnas
# pero también mantendremos las columnas "country" (país) y "year" (año)
final_keep_values = ["country_or_area", "year"] + keep_values

# Convierte los valores de la columna 'commodity transaction'
# en los nombres de nuestras nuevas columnas
# y conserva solo la columna 'quantity' como los nuevos valores
df_countries = pd.pivot_table(
    df,
    values="quantity",
    index=["country_or_area", "year"],
    columns="commodity_transaction",
).reset_index()[final_keep_values]

# Renombra las columnas para que sean más concisas
df_countries.columns = [
    "country",
    "year",
    "demand",
    "production",
    "imports",
    "exports",
    "hydro",
    "wind",
    "solar",
    "geothermal",
    "tide",
]

# Muestra los líderes en producción de energía primero
df_countries.sort_values(by="production", ascending=False)

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide
1062,China,2014,5219096.0,5649583.4,6750.0,18158.0,1064337.0,156078.0,15189.0,NaN,NaN
1061,China,2013,5016127.0,5431637.4,7438.0,18669.0,920291.0,141197.0,5564.0,NaN,NaN
1060,China,2012,4609729.0,4987553.0,6874.0,17653.0,872107.0,95978.0,NaN,NaN,NaN
1059,China,2011,4319132.0,4713019.0,6562.0,19307.0,698945.0,70331.0,NaN,NaN,NaN
5322,United States,2010,4153664.0,4378422.0,45083.0,19107.0,286333.0,95148.0,3934.0,17577.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2873,Lesotho,1996,335.0,NaN,335.0,NaN,NaN,NaN,NaN,NaN,NaN
2874,Lesotho,1997,395.0,NaN,395.0,NaN,NaN,NaN,NaN,NaN,NaN
2875,Lesotho,1998,385.0,NaN,385.0,NaN,NaN,NaN,NaN,NaN,NaN
3454,Namibia,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Podemos ver que nuestros datos ahora están en un formato mucho más fácil de usar. Hemos conservado solo la columna de cantidad y cada fila ahora representa un país en un año específico. Si tuviéramos datos para cada año de cada uno de los 243 países o áreas, esperaríamos tener 6075 filas, pero solo tenemos 5568. Esto tiene sentido, ya que algunos países dejaron de existir y la recolección de datos, en general, se ha vuelto mucho más fácil y consistente con el tiempo. Veamos para cuántos países tenemos datos en cada año:

In [ ]:
df_countries['year'].value_counts()

,count
year,
2014,229
2013,229
2012,229
2007,227
2011,226
2010,226
2009,226
2008,226
2006,226


Como era de esperarse, en los primeros años tenemos datos de menos países.

La verificación final que debemos hacer es si alguno de los valores que conservamos utiliza una "unidad" diferente. Un análisis rápido de los datos muestra que todos los valores que nos interesan están medidos en "Kilovatios-hora, millón", pero es posible que algunos valores pequeños estén medidos como "Kilovatios-hora, mil", por ejemplo. Veamos los valores únicos utilizados en nuestra lista keep_values:

In [ ]:
x = keep_values[0]
all_units = []

for value in keep_values:
    units_used = list(df[df.commodity_transaction == value]['unit'].unique())
    all_units += units_used
print(set(all_units))

{'Kilowatt-hours, million'}


¡Todo bien! Solo se usa una unidad. Así que hemos terminado con la preparación de los datos y podemos comenzar a explorar nuestro conjunto de datos en busca de información.

## Explorando el crecimiento de la producción de energía y las renovables

Como se mencionó, el equipo está interesado en analizar los países en función de su producción de energía renovable. Actualmente sabemos cuánta energía producen en total y cuánto de esto se debe a cada una de las opciones renovables disponibles. Comenzaremos agregando algunos datos complementarios y luego analizaremos nuestro conjunto de datos para identificar países y patrones interesantes.

### Ejercicio 4:

Agrega una nueva columna de resumen llamada renewable_percent, que indique el porcentaje de la producción total de energía que proviene de fuentes renovables.

**Pista:** Podrías notar que algunos valores aparecen como na, lo que significa "no disponible". Probablemente podamos asumir que estos valores son 0 (aunque esto no siempre es significativo; por ejemplo, si no tenemos datos de la URSS en 2014, no es porque todas sus plantas de energía estén apagadas). Puedes usar el método fillna de pandas para reemplazar los valores na con 0.

**Respuesta.**

In [ ]:
# Agrega una nueva columna de resumen llamada renewable_percent, que indique el porcentaje de la producción total de energía que proviene de fuentes renovables.
# Puedes usar el método fillna de pandas para reemplazar los valores na con 0.
print(df_countries.columns)
# Calcular el total de producción renovable
renewable_sources = ['hydro', 'wind', 'solar', 'geothermal', 'tide']
df_countries['renewable_total'] = df_countries[renewable_sources].sum(axis=1)

# Calcular el porcentaje de producción renovable
df_countries['renewable_percent'] = (df_countries['renewable_total'] / df_countries['production']) * 100

# Reemplazar NaN con 0
df_countries['renewable_percent'] = df_countries['renewable_percent'].fillna(0)
print(df_countries.head())

Index(['country', 'year', 'demand', 'production', 'imports', 'exports',
       'hydro', 'wind', 'solar', 'geothermal', 'tide'],
      dtype='object')
       country  year  demand  production  imports  exports  hydro  wind  \
0  Afghanistan  1990  1055.0      1128.0      NaN      NaN  764.0   NaN   
1  Afghanistan  1991   945.0      1015.0      NaN      NaN  690.0   NaN   
2  Afghanistan  1992   789.0       703.0    131.0      NaN  478.0   NaN   
3  Afghanistan  1993   780.0       695.0    130.0      NaN  475.0   NaN   
4  Afghanistan  1994   770.0       687.0    128.0      NaN  472.0   NaN   

   solar  geothermal  tide  renewable_total  renewable_percent  
0    NaN         NaN   NaN            764.0          67.730496  
1    NaN         NaN   NaN            690.0          67.980296  
2    NaN         NaN   NaN            478.0          67.994310  
3    NaN         NaN   NaN            475.0          68.345324  
4    NaN         NaN   NaN            472.0          68.704512  


### Ejercicio 5:

Considerando solo el año más reciente para el cual tenemos datos (2014), ¿qué 5 países produjeron la mayor proporción de su energía a partir de fuentes renovables y cuáles 5 países produjeron la menor proporción de su energía a partir de fuentes renovables?

**Pista:** Puedes usar el método sort_values de pandas para ordenar un DataFrame por una columna específica, ya sea de forma ascendente o descendente.

**Respuesta.**

In [ ]:
# Considerando solo el año más reciente para el cual tenemos datos (2014),
# ¿qué 5 países produjeron la mayor proporción de su energía a partir de fuentes renovables y
# cuáles 5 países produjeron la menor proporción de su energía a partir de fuentes renovables?
# Pista: Puedes usar el método sort_values de pandas para ordenar un DataFrame por una columna específica, ya sea de forma ascendente o descendente.
df_countries_2014 = df_countries[df_countries['year'] == 2014]
df_countries_2014 = df_countries_2014.sort_values(by='renewable_percent', ascending=False)
print('TOP 5: ',df_countries_2014.head(5))
print('\n\n-----------------------------------------\n\n')
print('TAIL 5: ',df_countries_2014.tail(5))

TOP 5:         country  year    demand  production  imports   exports     hydro  wind  \
2891   Lesotho  2014    783.48      515.20   271.20      2.92    515.20   NaN   
49     Albania  2014   7791.43     4724.43  3250.45    183.45   4724.43   NaN   
611     Bhutan  2014   2085.46     7003.86   187.37   4991.90   7003.36   NaN   
3948  Paraguay  2014  13432.00    55282.30      NaN  41400.10  55276.40   NaN   
2328   Iceland  2014  17475.00    18122.00      NaN       NaN  12873.00   8.0   

      solar  geothermal  tide  renewable_total  renewable_percent  
2891    NaN         NaN   NaN           515.20         100.000000  
49      NaN         NaN   NaN          4724.43         100.000000  
611     NaN         NaN   NaN          7003.36          99.992861  
3948    NaN         NaN   NaN         55276.40          99.989328  
2328    NaN      5238.0   NaN         18119.00          99.983446  


-----------------------------------------


TAIL 5:                country  year    demand  pro

### Pregunta:

¿Por qué crees que estamos viendo muchos países muy pequeños en ambas listas?

Los países muy pequeños no son particularmente representativos de la situación global de la energía renovable, por lo que tu equipo te pide que restrinjas tu análisis solo a los países que producen mucha energía.

### Ejercicio 6:

Repite el análisis anterior, pero solo considera los países en el 10% superior de producción total de energía.

**Pista:** Puedes filtrar un DataFrame con múltiples condiciones utilizando el símbolo &. Por ejemplo:

`df_countries[df_countries.year == 2014 & df_countries.wind > 0]`

Esto te proporcionaría un DataFrame con todos los países en 2014 que produjeron al menos algo de energía eólica.

**Respuesta.**

In [ ]:
df_countries_10_percent = df_countries_2014[df_countries['production'] > df_countries_2014['production'].quantile(0.9)]
df_countries_10_percent

<ipython-input-34-b85d9d8db506>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_countries_10_percent = df_countries_2014[df_countries['production'] > df_countries_2014['production'].quantile(0.9)]


,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide,renewable_total,renewable_percent
712,Brazil,2014,615629.0,590541.0,33778.0,3.0,373439.0,12211.0,16.0,NaN,NaN,385666.0,65.307235
937,Canada,2014,591137.0,656225.0,12808.0,58421.0,382574.0,22538.0,1756.0,NaN,16.0,406884.0,62.003733
4569,Spain,2014,259772.0,278750.0,12310.0,15716.0,42970.0,52013.0,13673.0,NaN,NaN,108656.0,38.979731
2523,Italy,2014,310535.0,279827.0,46747.0,3031.0,60256.0,15178.0,22306.0,5916.0,NaN,103656.0,37.042887
1062,China,2014,5219096.0,5649583.4,6750.0,18158.0,1064337.0,156078.0,15189.0,NaN,NaN,1235604.0,21.870710
5103,Turkey,2014,244706.0,251963.0,7953.0,2696.0,40645.0,8520.0,17.0,2364.0,NaN,51546.0,20.457766
1939,Germany,2014,550064.0,627795.0,40435.0,74320.0,25444.0,57357.0,36056.0,98.0,NaN,118955.0,18.948064
3295,Mexico,2014,289521.0,301496.0,2124.0,2653.0,38893.0,6426.0,221.0,6000.0,NaN,51540.0,17.094754
4169,Russian Federation,2014,983691.0,1064207.0,6623.0,14671.0,177141.0,96.0,160.0,455.0,NaN,177852.0,16.712162
1791,France,2014,464250.0,562776.0,7873.0,75063.0,68626.0,17249.0,5909.0,NaN,481.0,92265.0,16.394622


In [ ]:
# Repite el análisis anterior, pero solo considera los países en el 10% superior de producción total de energía.
# Pista: Puedes filtrar un DataFrame con múltiples condiciones utilizando el símbolo &. Por ejemplo:
# df_countries[df_countries.year == 2014 & df_countries.wind > 0]
# Esto te proporcionaría un DataFrame con todos los países en 2014 que produjeron al menos algo de energía eólica.
df_countries_10_percent = df_countries_2014[df_countries['renewable_percent'] > df_countries_2014['renewable_percent'].quantile(0.1)]
df_countries_10_percent = df_countries_10_percent.sort_values(by='renewable_percent', ascending=False)
print('TOP 5: ',df_countries_10_percent.head(5))
print('\n\n-----------------------------------------\n\n')
print('TAIL 5: ',df_countries_10_percent.tail(5))

# df_countries[df_countries.year == 2014 & df_countries.wind > 0] tener en cuenta

TOP 5:         country  year    demand  production  imports   exports     hydro  wind  \
2891   Lesotho  2014    783.48      515.20   271.20      2.92    515.20   NaN   
49     Albania  2014   7791.43     4724.43  3250.45    183.45   4724.43   NaN   
611     Bhutan  2014   2085.46     7003.86   187.37   4991.90   7003.36   NaN   
3948  Paraguay  2014  13432.00    55282.30      NaN  41400.10  55276.40   NaN   
2328   Iceland  2014  17475.00    18122.00      NaN       NaN  12873.00   8.0   

      solar  geothermal  tide  renewable_total  renewable_percent  
2891    NaN         NaN   NaN           515.20         100.000000  
49      NaN         NaN   NaN          4724.43         100.000000  
611     NaN         NaN   NaN          7003.36          99.992861  
3948    NaN         NaN   NaN         55276.40          99.989328  
2328    NaN      5238.0   NaN         18119.00          99.983446  


-----------------------------------------


TAIL 5:                  country  year    demand  p

<ipython-input-24-c165713bf2d9>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_countries_10_percent = df_countries_2014[df_countries['renewable_percent'] > df_countries_2014['renewable_percent'].quantile(0.1)]


_________

Por supuesto, tu equipo también está interesado en analizar los cambios en la energía renovable a lo largo del tiempo. Veamos los 5 países con los mayores y menores cambios en la proporción de energía renovable producida en 2014 en comparación con 1990.

### Ejercicio 7:

Agrega una nueva columna a tu DataFrame que muestre la diferencia en el porcentaje de producción de energía renovable entre 2014 y 1990. ¿Cuáles son los 5 países con los mayores y menores cambios? ¿Qué observas sobre estos países? Realiza este análisis tanto con todos los países como solo con aquellos en el 10% superior de producción total de energía.

**Pista:** Puedes usar el método pivot() nuevamente para crear un DataFrame que tenga 1990 y 2014 como columnas y renewable_percent como valores, para ayudar con esto utilizando el siguiente código:

```
renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

**Respuesta.**

In [ ]:
# Agrega una nueva columna a tu DataFrame que muestre la diferencia en el porcentaje de producción de energía renovable entre 2014 y 1990.
# ¿Cuáles son los 5 países con los mayores y menores cambios?
# ¿Qué observas sobre estos países?
# Realiza este análisis tanto con todos los países como solo con aquellos en el 10% superior de producción total de energía.
# Pista: Puedes usar el método pivot() nuevamente para crear un DataFrame que tenga 1990 y 2014 como columnas y
# renewable_percent como valores, para ayudar con esto utilizando el siguiente código:
# renewable_change = pd.pivot_table(
#    df_countries, values="renewable_percent", index=["country"], columns="year",
# ).reset_index()[["country", 1990, 2014]]

# Primero, creamos la tabla pivote para obtener los porcentajes de energía renovable de 1990 y 2014
renewable_change = pd.pivot_table(
    df_countries,
    values="renewable_percent",
    index=["country"],
    columns="year"
).reset_index()[["country", 1990, 2014]]

# Ahora, agregamos una nueva columna que calcule la diferencia en porcentaje
renewable_change['difference'] = renewable_change[2014] - renewable_change[1990]

# Encontramos los 5 países con los mayores cambios
top_5_increase = renewable_change.nlargest(5, 'difference')

# Encontramos los 5 países con los menores cambios
top_5_decrease = renewable_change.nsmallest(5, 'difference')

# Observaciones sobre los países
print("Top 5 países con mayores cambios:")
print(top_5_increase)

print("\nTop 5 países con menores cambios:")
print(top_5_decrease)

# Ahora, realizamos el análisis solo con el 10% superior de producción total de energía
# Primero, calculamos la producción total de energía
renewable_total = df_countries.groupby('country')['production'].sum().reset_index()

# Obtenemos el 10% superior
top_10_percent_threshold = renewable_total['production'].quantile(0.1)
top_10_percent_countries = renewable_total[renewable_total['production'] >= top_10_percent_threshold]['country']

# Filtramos el DataFrame de cambios para solo incluir estos países
top_10_percent_changes = renewable_change[renewable_change['country'].isin(top_10_percent_countries)]

# Encontramos los 5 países con los mayores cambios en el 10% superior
top_5_increase_top_10 = top_10_percent_changes.nlargest(5, 'difference')

# Encontramos los 5 países con los menores cambios en el 10% superior
top_5_decrease_top_10 = top_10_percent_changes.nsmallest(5, 'difference')

print("\nTop 5 países con mayores cambios (10% superior de producción):")
print(top_5_increase_top_10)

print("\nTop 5 países con menores cambios (10% superior de producción):")
print(top_5_decrease_top_10)


Top 5 países con mayores cambios:
year        country  1990        2014  difference
123         Lesotho   0.0  100.000000  100.000000
148         Namibia   0.0   99.132176   99.132176
88        Greenland   0.0   68.347538   68.347538
187    Sierra Leone   0.0   65.356945   65.356945
77    French Guiana   0.0   60.549451   60.549451

Top 5 países con menores cambios:
year                  country       1990       2014  difference
197                 Sri Lanka  99.841270  37.764953  -62.076316
178                    Rwanda  97.660819  39.088427  -58.572392
98                   Honduras  91.254919  37.322717  -53.932202
206                  Suriname  85.859991  36.479170  -49.380820
228   United Rep. of Tanzania  89.686924  41.936003  -47.750922

Top 5 países con mayores cambios (10% superior de producción):
year        country  1990        2014  difference
123         Lesotho   0.0  100.000000  100.000000
148         Namibia   0.0   99.132176   99.132176
88        Greenland   0.0   68.34

In [ ]:
top_10_percent_changes

year,country,1990,2014,difference
0,Afghanistan,67.730496,85.323549,17.593053
1,Albania,87.613388,100.000000,12.386612
2,Algeria,0.838301,0.395380,-0.442921
3,American Samoa,0.000000,0.700882,0.700882
4,Andorra,100.000000,89.432177,-10.567823
...,...,...,...,...
235,Viet Nam,61.579913,42.393467,-19.186445
237,Yemen,NaN,0.000000,NaN
240,"Yugoslavia, SFR (former)",24.237380,NaN,NaN
241,Zambia,99.485266,97.163022,-2.322243


### Ejercicio 8:

Tu equipo también está interesado en los países que ahora están produciendo mucha más energía que hace 25 años. ¿Cuáles son los 10 países con mayor y menor crecimiento en términos de:

* Total power
* Renewable power

Ten en cuenta que, dado que muchos países producían poca o ninguna energía renovable en 1990, realizar un cálculo básico de crecimiento mostrará que muchos países tienen un crecimiento "infinito" (representado como inf en pandas). Para evitar esto, restringe tus resultados a los países que produjeron al menos 1,000 unidades de energía renovable en 1990 para el análisis de crecimiento renovable y al menos 1,000 unidades de energía total para el análisis de crecimiento total.

**Pista:** Suponiendo que agregaste una columna llamada renewable_total, puedes usar los siguientes pivotes para generar tablas similares a las anteriores tanto para el crecimiento renovable como para el crecimiento total::

```
renewable_growth = pd.pivot_table(
    df_countries, values="renewable_total", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

```
total_growth = pd.pivot_table(
    df_countries, values="production", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

**Respuesta.**

In [ ]:
# Tu equipo también está interesado en los países que ahora están produciendo mucha más energía que hace 25 años.
# ¿Cuáles son los 10 países con mayor y menor crecimiento en términos de:
# Total power
# Renewable power
# Ten en cuenta que, dado que muchos países producían poca o ninguna energía renovable en 1990, realizar un cálculo básico de crecimiento mostrará que muchos países tienen un crecimiento "infinito" (representado como inf en pandas). Para evitar esto, restringe tus resultados a los países que produjeron al menos 1,000 unidades de energía renovable en 1990 para el análisis de crecimiento renovable y al menos 1,000 unidades de energía total para el análisis de crecimiento total.
# Pista: Suponiendo que agregaste una columna llamada renewable_total, puedes usar los siguientes pivotes para generar tablas similares a
#las anteriores tanto para el crecimiento renovable como para el crecimiento total::
# renewable_growth = pd.pivot_table(
#    df_countries, values="renewable_total", index=["country"], columns="year",
# ).reset_index()[["country", 1990, 2014]]
# total_growth = pd.pivot_table(
#    df_countries, values="production", index=["country"], columns="year",
# ).reset_index()[["country", 1990, 2014]]

# Crear tablas pivot para el crecimiento renovable y total
renewable_growth = pd.pivot_table(
    df_countries, values="renewable_total", index=["country"], columns="year"
).reset_index()[["country", 1990, 2014]]

total_growth = pd.pivot_table(
    df_countries, values="production", index=["country"], columns="year"
).reset_index()[["country", 1990, 2014]]

# Filtrar países con al menos 1,000 unidades de energía renovable en 1990
renewable_growth = renewable_growth[renewable_growth[1990] >= 1000]

# Filtrar países con al menos 1,000 unidades de energía total en 1990
total_growth = total_growth[total_growth[1990] >= 1000]

# Calcular el crecimiento
renewable_growth['growth'] = (renewable_growth[2014] - renewable_growth[1990]) / renewable_growth[1990]
total_growth['growth'] = (total_growth[2014] - total_growth[1990]) / total_growth[1990]

# Obtener los 10 países con mayor y menor crecimiento en energía renovable
top_10_renewable_growth = renewable_growth.nlargest(10, 'growth')
bottom_10_renewable_growth = renewable_growth.nsmallest(10, 'growth')

# Obtener los 10 países con mayor y menor crecimiento en energía total
top_10_total_growth = total_growth.nlargest(10, 'growth')
bottom_10_total_growth = total_growth.nsmallest(10, 'growth')

# Mostrar los resultados
print("Top 10 países con mayor crecimiento en energía renovable:")
print(top_10_renewable_growth)
print('\n ----------------------------------------------------------- \n')
print("\nBottom 10 países con menor crecimiento en energía renovable:")
print(bottom_10_renewable_growth)
print('\n ----------------------------------------------------------- \n')
print("\nTop 10 países con mayor crecimiento en energía total:")
print(top_10_total_growth)
print('\n ----------------------------------------------------------- \n')
print("\nBottom 10 países con menor crecimiento en energía total:")
print(bottom_10_total_growth)


Top 10 países con mayor crecimiento en energía renovable:
year         country      1990        2014     growth
235         Viet Nam    5371.0    61780.00  10.502513
43             China  126720.0  1235604.00   8.750663
147          Myanmar    1193.0     8828.84   6.400536
227   United Kingdom    7198.0    44835.00   5.228814
87            Greece    1999.0    12088.00   5.047024
24            Bhutan    1557.0     7003.36   3.497983
196            Spain   26204.0   108656.00   3.146543
32          Bulgaria    1878.0     7746.00   3.124601
100          Iceland    4504.0    18119.00   3.022869
131         Malaysia    3982.0    13615.00   2.419136

 ----------------------------------------------------------- 


Bottom 10 países con menor crecimiento en energía renovable:
year                country      1990      2014    growth
206                Suriname    1111.2     795.1 -0.284467
116   Korea, Dem.Ppl's.Rep.   15600.0   13000.0 -0.166667
208                  Sweden   73039.0   75153.0 

### Ejercicio 9:

Finalmente, tu equipo quiere una etiqueta fácil de leer para cada país basada en el crecimiento total. Te han dado la siguiente especificación para etiquetar a los países:

* zero or negative growth = "No growth"
* 1% -100% growth = "Growing"
* over 100% growth = "Growing fast"
* NaN (if the data from 1990 or 2014 is NaN) = "Not Applicable"

Calcula la etiqueta para cada país utilizando el método apply() para mayor eficiencia.

**Pista:** Puedes verificar si el valor de la variable x es NaN de la siguiente manera:

```
import numpy as np
np.isnan(x)
```

**Respuesta.**

In [ ]:
# Finalmente, tu equipo quiere una etiqueta fácil de leer para cada país basada en el crecimiento total.
# Te han dado la siguiente especificación para etiquetar a los países:
# zero or negative growth = "No growth"
# 1% -100% growth = "Growing"
# over 100% growth = "Growing fast"
# NaN (if the data from 1990 or 2014 is NaN) = "Not Applicable"
# Calcula la etiqueta para cada país utilizando el método apply() para mayor eficiencia.
# Pista: Puedes verificar si el valor de la variable x es NaN de la siguiente manera:
# import numpy as np
# np.isnan(x)

def label_growth(current_value, base_value):
    if np.isnan(current_value) or np.isnan(base_value):
        return "Not Applicable"

    growth = current_value - base_value  # Cálculo del crecimiento
    percentage_growth = (growth / base_value) * 100 if base_value != 0 else float('inf')  # Evitar división por cero

    if percentage_growth < 0:
        return "No growth"
    elif 0 <= percentage_growth <= 100:
        return "Growing"
    else:  # percentage_growth > 100
        return "Growing fast"

# Merge df_countries with total_growth to get production values for 1990
df_countries = pd.merge(df_countries, total_growth[['country', 1990]], on='country', how='left')
# Rename the 1990 column to production_1990
df_countries = df_countries.rename(columns={1990: 'production_1990'})

# Assuming 'production_1990' is the column containing production values for 1990
df_countries['result'] = df_countries.apply(lambda row: label_growth(row['production'], row['production_1990']), axis=1)
df_countries['result'].value_counts()

,count
result,
Not Applicable,2934
Growing,1635
Growing fast,782
No growth,217


## Importadores y exportadores de energía más grandes

El último aspecto que tu equipo quiere analizar son las importaciones y exportaciones de energía por país.

### Ejercicio 10:

Tu equipo quiere saber:

* ¿Qué países han importado y exportado la mayor cantidad de energía en total?
* ¿Qué países han importado el mayor porcentaje de su demanda y exportado el mayor porcentaje de su producción?

Realiza el análisis para todos los países y también para solo aquellos con producción total dentro del 10% superior.

**Respuesta.**

In [ ]:
# Tu equipo quiere saber:
# ¿Qué países han importado y exportado la mayor cantidad de energía en total?
# ¿Qué países han importado el mayor porcentaje de su demanda y exportado el mayor porcentaje de su producción?
#Realiza el análisis para todos los países y también para solo aquellos con producción total dentro del 10% superior.

# Calcular el total de producción para determinar el 10% superior
top_10_percent_threshold = df_countries['production'].quantile(0.1)

# Filtrar países con producción total dentro del 10% superior
top_producers = df_countries[df_countries['production'] >= top_10_percent_threshold]

# Análisis para todos los países
# 1. Países que han importado y exportado la mayor cantidad de energía
max_imports = df_countries.nlargest(10, 'imports')[['country', 'imports']]
max_exports = df_countries.nlargest(10, 'exports')[['country', 'exports']]

# 2. Países que han importado el mayor porcentaje de su demanda
df_countries['import_percentage'] = df_countries['imports'] / df_countries['demand'] * 100
top_import_percentage = df_countries.nlargest(10, 'import_percentage')[['country', 'import_percentage']]

# 3. Países que han exportado el mayor porcentaje de su producción
df_countries['export_percentage'] = df_countries['exports'] / df_countries['production'] * 100
top_export_percentage = df_countries.nlargest(10, 'export_percentage')[['country', 'export_percentage']]

# Análisis para los países dentro del 10% superior
# 1. Países que han importado y exportado la mayor cantidad de energía
max_imports_top = top_producers.nlargest(10, 'imports')[['country', 'imports']]
max_exports_top = top_producers.nlargest(10, 'exports')[['country', 'exports']]

# 2. Países que han importado el mayor porcentaje de su demanda
top_producers['import_percentage'] = top_producers['imports'] / top_producers['imports'] * 100
top_import_percentage_top = top_producers.nlargest(10, 'import_percentage')[['country', 'import_percentage']]

# 3. Países que han exportado el mayor porcentaje de su producción
top_producers['export_percentage'] = top_producers['exports'] / top_producers['production'] * 100
top_export_percentage_top = top_producers.nlargest(10, 'export_percentage')[['country', 'export_percentage']]

# Mostrar resultados
print("Países que han importado la mayor cantidad de energía:")
print(max_imports)

print("\nPaíses que han exportado la mayor cantidad de energía:")
print(max_exports)

print("\nPaíses que han importado el mayor porcentaje de su demanda:")
print(top_import_percentage)

print("\nPaíses que han exportado el mayor porcentaje de su producción:")
print(top_export_percentage)

print("\n--- Análisis para países en el 10% superior ---")

print("\nPaíses que han importado la mayor cantidad de energía (10% superior):")
print(max_imports_top)

print("\nPaíses que han exportado la mayor cantidad de energía (10% superior):")
print(max_exports_top)

print("\nPaíses que han importado el mayor porcentaje de su demanda (10% superior):")
print(top_import_percentage_top)

print("\nPaíses que han exportado el mayor porcentaje de su producción (10% superior):")
print(top_export_percentage_top)


Países que han importado la mayor cantidad de energía:
            country  imports
5325  United States  70355.0
5326  United States  66511.0
5324  United States  59257.0
5320  United States  57019.0
1930        Germany  56861.0
5323  United States  52301.0
5306  United States  52230.0
5321  United States  52191.0
2511          Italy  51519.0
2512          Italy  51486.0

Países que han exportado la mayor cantidad de energía:
      country  exports
1779   France  80739.0
1791   France  75063.0
1939  Germany  74320.0
1780   France  73373.0
1777   France  73174.0
1778   France  72861.0
1772   France  72701.0
1773   France  72428.0
1783   France  71863.0
1938  Germany  71415.0

Países que han importado el mayor porcentaje de su demanda:
         country  import_percentage
2981  Luxembourg         116.314920
2980  Luxembourg         115.092540
2979  Luxembourg         114.171497
2974  Luxembourg         112.104412
2978  Luxembourg         111.985968
2973  Luxembourg         110.931942
2983

<ipython-input-28-e56f05401c2e>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_producers['import_percentage'] = top_producers['imports'] / top_producers['imports'] * 100
<ipython-input-28-e56f05401c2e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_producers['export_percentage'] = top_producers['exports'] / top_producers['production'] * 100


## Escribiendo datos de resumen específicos por país en el disco

Tu equipo está encantado de que hayas logrado interpretar los datos y extraer algunos conocimientos. Quieren explorar los datos por sí mismos, pero todas las herramientas que tienen están diseñadas para analizar datos de un solo país a la vez. Te han pedido que crees archivos CSV separados para cada país, utilizando el nombre del país como nombre del archivo, con un máximo de 25 filas por archivo (una por año) y columnas para importaciones, exportaciones, etc.

Para hacerlo, usamos la función to_csv en un DataFrame dado para escribirlo en un archivo. Creamos un nuevo directorio llamado "output_csvs" en nuestro directorio de trabajo para no saturar nuestro espacio de trabajo con 243 archivos CSV. Luego, ejecutamos el siguiente código para escribir nuestros datos en el disco:

In [ ]:
# import os

OUTPUT_DIRECTORY = "output_csvs"

if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

for country in df_countries['country'].unique():
    country_df = df_countries[df_countries.country == country].drop(columns='country')
    country_df.to_csv(f"{OUTPUT_DIRECTORY}/{country}.csv")

## Conclusiones

Observamos varias tendencias interesantes en la industria energética global. En concreto, vimos que muchos países dependen cada vez más de las energías renovables, pero que algunos de los países con una demanda creciente se ven obligados a recurrir a fuentes no renovables para mantenerse al día.

También notamos que, en contra de nuestras expectativas de que algunos países serían "importadores netos" y otros "exportadores netos" de energía, muchos países en realidad importan y exportan grandes cantidades de energía.

## Lecciones aprendidas

En este caso, cubrimos más funciones de pandas y practicamos con las características que vimos previamente. Específicamente, aprendimos cómo:

* Usar el método apply() en pandas con funciones integradas, funciones personalizadas y funciones anónimas.
* Trabajar con grandes conjuntos de datos y explorarlos utilizando coincidencia básica de cadenas para encontrar columnas interesantes y reformatear los resultados en formatos más convenientes.
* Pivotar entre formatos amplios y estrechos.
* Graficar líneas básicas.
* Dividir un conjunto de datos grande en partes más pequeñas y escribirlas en el disco.

Aunque aprenderás funcionalidades más avanzadas en casos futuros, estos conceptos básicos se usarán una y otra vez, por lo que te recomendamos volver a este caso como material de referencia siempre que lo necesites.